In [1]:
import matplotlib.pyplot as plt
import pickle 
import numpy as np

from moviepy.editor import VideoClip, AudioFileClip

In [2]:
## parameters 
simulation_timestamps_per_second = 1000
video_frames_per_second = 30
audio_sample_rate = 44100 # Hz
speedup_rate=4

In [3]:
with open('trajectories.pkl', 'rb') as f:
    trajectories = pickle.load(f)[0]

In [4]:
# I want to make a smallest square region which contains all the trajectories
#   which is centered at origin. Also I set margin

margin = .5

xmax, xmin = np.max(trajectories[:,:,0]), np.min(trajectories[:,:,0])
ymax, ymin = np.max(trajectories[:,:,1]), np.min(trajectories[:,:,1])

width, height = xmax-xmin, ymax-ymin 
if width > height:
    ymin = ymin - (width-height)/2 
    ymax = ymax + (width-height)/2
else:
    xmin = xmin - (height-width)/2
    xmax = xmax + (height-width)/2

In [5]:
def generate_frame(trajectories,
                   video_frame_number,
                   trace_window_in_seconds=10,
                   simulation_timestamps_per_second=simulation_timestamps_per_second,
                   video_frames_per_second=video_frames_per_second, 
                   speedup_rate=speedup_rate, 
                   showtitle=False):
    
    fig, ax = plt.subplots(figsize=(20,20))

    fig.patch.set_facecolor('k')
    ax.set_axis_off()
    
    margin = 1

    ax.set_xlim(xmin - margin, xmax + margin)
    ax.set_ylim(ymin - margin, ymax + margin)
    ax.set_aspect('equal')
    
    # Calculate the corresponding simulation timestamp for the current frame
    simulation_timestamp = int(video_frame_number * simulation_timestamps_per_second / video_frames_per_second)
    
    trace_window_in_simulation_timestamps = trace_window_in_seconds * simulation_timestamps_per_second
    window_startpt_in_simstamps = max(0, simulation_timestamp - trace_window_in_simulation_timestamps) 

    lw = 3 # linewidth    
    # Plot the trajectories up to the current simulation timestamp    
    for idx, trajectory in enumerate(trajectories):
        ax.plot(
            trajectory[window_startpt_in_simstamps:simulation_timestamp, 0],
            trajectory[window_startpt_in_simstamps:simulation_timestamp, 1],
            c='rgb'[idx], 
            linewidth=lw
            )
    ## If window_startpt_in_simstamps is not 0, the remaining trace will be plotted more transparently
    if window_startpt_in_simstamps != 0:
        for idx, trajectory in enumerate(trajectories):
            ax.plot(
                trajectory[:window_startpt_in_simstamps, 0],
                trajectory[:window_startpt_in_simstamps, 1],
                c='rgb'[idx], 
                linewidth=lw, alpha=0.8
                )
    
    psize = 250
    # Plot the current positions
    ax.scatter(trajectories[0, simulation_timestamp, 0],
               trajectories[0, simulation_timestamp, 1], c='r', s=psize, marker='.')
    ax.scatter(trajectories[1, simulation_timestamp, 0],
               trajectories[1, simulation_timestamp, 1], c='g', s=psize, marker='.')
    ax.scatter(trajectories[2, simulation_timestamp, 0],
               trajectories[2, simulation_timestamp, 1], c='b', s=psize, marker='.')
    
    if showtitle:    
        ax.set_title(f"Frame {video_frame_number//speedup_rate}, Time {simulation_timestamp/simulation_timestamps_per_second/speedup_rate:.2f}")
    
    ax.set_axis_off()
    
    fig.canvas.draw()
    frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    plt.close(fig)
    return frame


In [9]:
duration  = int(len(trajectories[0]) / simulation_timestamps_per_second//speedup_rate)

In [10]:
def make_frame(t):
    frame_number = int(t * video_frames_per_second)
    
    return generate_frame(trajectories,
                           frame_number * speedup_rate,
                           #simulation_timestamps_per_second, #video_frames_per_second
                           )

video_clip = VideoClip(make_frame, duration=duration)

In [12]:
audio_wav_fast = AudioFileClip('./3body_audio_fast.wav')
final_clip = video_clip.set_audio(audio_wav_fast)

In [17]:
final_clip.write_videofile("./3body_simulation_with_noise.mp4", fps=video_frames_per_second)